In [1]:
import http.client
import json
import pandas as pd
from pandas.io.json import json_normalize
import pickle
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

month = 4
day = 14

conn = http.client.HTTPSConnection("api.sportradar.us")

conn.request("GET",
             "/mlb/trial/v6.5/en/games/2019/{0}/{1}/boxscore."
             "json?api_key=bw84ac36vu34rdk5vkkh4psx".format(month, day))

res = conn.getresponse()
data = res.read()

baseball_data = []
baseball_data.append(data.decode("utf-8"))

baseball_json = []
baseball_json.append(json.loads(baseball_data[0]))
del baseball_json[0]['_comment']

baseball_normal = json_normalize(baseball_json)
baseball_normal = baseball_normal.drop(columns=
                                       ['league.alias', 'league.date', 'league.id', 'league.name'])

baseball_normal_14 = json_normalize(baseball_normal.iloc[0, 0])

with open('outfile_april_14_pre', 'wb') as fp:
    pickle.dump(baseball_normal_14, fp)

In [2]:
with open('baseball2018', 'rb') as fp:
    baseball_data = pickle.load(fp)  # Load up the 2018 data

del baseball_data[-1]  # Last day, no games
del baseball_data[108]  # All star break, no games
del baseball_data[108]  # All star break, no games
del baseball_data[108]  # All star break, no games
del baseball_data[30]  # April 31 does not exist
del baseball_data[91]  # June 31 does not exist

baseball_json = []

for days in baseball_data:
    baseball_json.append(json.loads(days))

for days in baseball_json:
    del days['_comment']  # API pull had weird last key value pair, delete

baseball_json_norm = json_normalize(baseball_json)
baseball_json_norm = baseball_json_norm.drop(
    columns=['league.alias', 'league.date', 'league.id', 'league.name'])

data = json_normalize(baseball_json_norm.iloc[0, 0])
for rows in range(1, 180):
    data = data.append(json_normalize(baseball_json_norm.iloc[rows, 0]),
                       ignore_index=True, sort=False)

data['home.win.pct'] = data['game.home.win'] / (data['game.home.win'] + data['game.home.loss'])
data['away.win.pct'] = data['game.away.win'] / (data['game.away.win'] + data['game.away.loss'])
data['total.runs'] = data['game.home.runs'] + data['game.away.runs']  # Response variable!

with open('outfile_april_12_pre', 'rb') as fp:
    baseball_data1 = pickle.load(fp)

data_all_cols = list(data.columns.values)
data_good_cols = list(baseball_data1.columns.values)
data_good_cols.append('home.win.pct')
data_good_cols.append('away.win.pct')
data_good_cols.append('total.runs')
data = data.drop(list(set(data_all_cols) - set(data_good_cols)), axis=1)
# data contains only the 67 variables available before today's MLB games start

# Must drop additional variables will not know before game starts
data = data.drop(['game.away.errors', 'game.away.hits', 'game.away.runs',
                  'game.home.errors', 'game.home.hits', 'game.home.runs'], axis=1)

# Must drop unnecessary variables
data = data.drop(['game.away.id', 'game.away.market', 'game.away.name',
                  'game.away.probable_pitcher.first_name',
                  'game.away.probable_pitcher.jersey_number',
                  'game.away.probable_pitcher.last_name',
                  'game.away.probable_pitcher.preferred_name',
                  'game.away_team', 'game.coverage',
                  'game.game_number', 'game.home.id',
                  'game.home.market', 'game.home.name',
                  'game.home.probable_pitcher.first_name',
                  'game.home.probable_pitcher.jersey_number',
                  'game.home.probable_pitcher.last_name',
                  'game.home.probable_pitcher.preferred_name',
                  'game.home_team', 'game.rescheduled',
                  'game.scheduled', 'game.status',
                  'game.venue.address', 'game.venue.country',
                  'game.venue.market', 'game.venue.state',
                  'game.venue.zip', 'game.weather.forecast.obs_time',
                  'game.id', 'game.venue.city', 'game.venue.id',
                  'game.double_header', 'game.venue.location.lat',
                  'game.venue.location.lng'], axis=1)    # Might need these: , 
                 #'game.away.probable_pitcher.id',
                 #'game.home.probable_pitcher.id'

data.drop_duplicates(inplace=True)
data = data.drop([373])  # Pesky row

# Pitcher records set to 0 if NaN
data['game.away.probable_pitcher.loss'].fillna(0, inplace=True)
data['game.away.probable_pitcher.win'].fillna(0, inplace=True)
data['game.home.probable_pitcher.loss'].fillna(0, inplace=True)
data['game.home.probable_pitcher.win'].fillna(0, inplace=True)

# Missing values replaced with "unknown" for text values
data['game.away.probable_pitcher.id'].fillna('unknown', inplace=True)   # Might need to comment out
data['game.home.probable_pitcher.id'].fillna('unknown', inplace=True)    # Might need to comment out
data['game.weather.forecast.condition'].fillna('unknown', inplace=True)
data['game.weather.forecast.wind.direction'].fillna('unknown', inplace=True)
data['game.venue.field_orientation'].fillna('unknown', inplace=True)
data['game.venue.stadium_type'].fillna('unknown', inplace=True)

# Weather data simplified
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("moderate snow", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("heavy snow", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("moderate or heavy snow showers", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("moderate or snow showers", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("patchy snow", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("patchy light snow", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("light snow", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("light snow, mist", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("squalls", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("snow, mist", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("light sleet", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("light drizzle", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("patchy rain possible", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Light rain", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("patchy light rain", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("moderate rain", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("light rain shower", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("patchy light drizzle", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("light rain, mist", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("light drizzle, mist", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Shower in Vicinity", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("overcast", "Cloudy", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Thundery outbreaks possible", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("rain shower", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("thunderstorm in vicinity", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("heavy rain", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("torrential rain shower", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("rain with thunderstorm, mist", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("thunderstorm", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("thunderstorm, haze", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("heavy rain, mist", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Partly cloudy", "Cloudy", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Patchy rain", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Rain, Mist", "Mist", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Torrential Rain", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Moderate or Storm", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Patchy rain with thunder", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("rain with storm", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("storm, rain", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("storm with storm", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Storm, Mist", "Mist", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Rain with thunder", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Blowing Widespread Dust", "Haze", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Smoke", "Haze", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Storm, Haze", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Haze, Smoke", "Haze", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Storm, Fog", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Haze, Haze", "Haze", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Fog", "Mist", case=False)

data.dropna(axis=0, inplace=True)

with open('model_data', 'wb') as fp:
    pickle.dump(data, fp)

In [3]:
with open('outfile_april_14_pre', 'rb') as fp:
    data = pickle.load(fp)  # Load up the 2018 data

data['home.win.pct'] = data['game.home.win'] / (data['game.home.win'] + data['game.home.loss'])
data['away.win.pct'] = data['game.away.win'] / (data['game.away.win'] + data['game.away.loss'])
data['total.runs'] = data['game.home.runs'] + data['game.away.runs']  # Response variable!

with open('outfile_april_12_pre', 'rb') as fp:
    baseball_data1 = pickle.load(fp)

data_all_cols = list(data.columns.values)
data_good_cols = list(baseball_data1.columns.values)
data_good_cols.append('home.win.pct')
data_good_cols.append('away.win.pct')
data_good_cols.append('total.runs')
data = data.drop(list(set(data_all_cols) - set(data_good_cols)), axis=1)
# data contains only the 67 variables available before today's MLB games start

# Must drop additional variables will not know before game starts
data = data.drop(['game.away.errors', 'game.away.hits', 'game.away.runs',
                  'game.home.errors', 'game.home.hits', 'game.home.runs'], axis=1)

# Must drop unnecessary variables
data = data.drop(['game.away.id', 'game.away.market', 'game.away.name',
                  'game.away.probable_pitcher.first_name',
                  'game.away.probable_pitcher.jersey_number',
                  'game.away.probable_pitcher.last_name',
                  'game.away.probable_pitcher.preferred_name',
                  'game.away_team', 'game.coverage',
                  'game.game_number', 'game.home.id',
                  'game.home.market', 'game.home.name',
                  'game.home.probable_pitcher.first_name',
                  'game.home.probable_pitcher.jersey_number',
                  'game.home.probable_pitcher.last_name',
                  'game.home.probable_pitcher.preferred_name',
                  'game.home_team', 
                  'game.scheduled', 'game.status',
                  'game.venue.address', 'game.venue.country',
                  'game.venue.market', 'game.venue.state',
                  'game.venue.zip', 'game.weather.forecast.obs_time',
                  'game.id', 'game.venue.city', 'game.venue.id',
                  'game.double_header', 'game.venue.location.lat',
                  'game.venue.location.lng'], axis=1)    # might need to add 'game.rescheduled'

#, 
#                 'game.away.probable_pitcher.id',
#                 'game.home.probable_pitcher.id'

# Pitcher records set to 0 if NaN
data['game.away.probable_pitcher.loss'].fillna(0, inplace=True)
data['game.away.probable_pitcher.win'].fillna(0, inplace=True)
data['game.home.probable_pitcher.loss'].fillna(0, inplace=True)
data['game.home.probable_pitcher.win'].fillna(0, inplace=True)

# Missing values replaced with "unknown" for text values
data['game.away.probable_pitcher.id'].fillna('unknown', inplace=True) # Might need to comment out
data['game.home.probable_pitcher.id'].fillna('unknown', inplace=True) # Might need to comment out
data['game.weather.forecast.condition'].fillna('unknown', inplace=True)
data['game.weather.forecast.wind.direction'].fillna('unknown', inplace=True)
data['game.venue.field_orientation'].fillna('unknown', inplace=True)
data['game.venue.stadium_type'].fillna('unknown', inplace=True)

# Weather data simplified
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("moderate snow", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("heavy snow", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("moderate or heavy snow showers", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("moderate or snow showers", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("patchy snow", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("patchy light snow", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("light snow", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("light snow, mist", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("squalls", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("snow, mist", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("light sleet", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("light drizzle", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("patchy rain possible", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Light rain", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("patchy light rain", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("moderate rain", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("light rain shower", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("patchy light drizzle", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("light rain, mist", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("light drizzle, mist", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Shower in Vicinity", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("overcast", "Cloudy", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Thundery outbreaks possible", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("rain shower", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("thunderstorm in vicinity", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("heavy rain", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("torrential rain shower", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("rain with thunderstorm, mist", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("thunderstorm", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("thunderstorm, haze", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("heavy rain, mist", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Partly cloudy", "Cloudy", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Patchy rain", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Rain, Mist", "Mist", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Torrential Rain", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Moderate or Storm", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Patchy rain with thunder", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("rain with storm", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("storm, rain", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("storm with storm", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Storm, Mist", "Mist", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Rain with thunder", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Blowing Widespread Dust", "Haze", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Smoke", "Haze", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Storm, Haze", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Haze, Smoke", "Haze", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Storm, Fog", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Haze, Haze", "Haze", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Fog", "Mist", case=False)

data.dropna(axis=0, inplace=True)

with open('model_predict_data', 'wb') as fp:
    pickle.dump(data, fp)

In [4]:
with open('model_data', 'rb') as fp:
    model_data = pickle.load(fp)  # Load up the 2018 modeling data
    
with open('model_predict_data', 'rb') as fp:
    model_predict_data = pickle.load(fp)  # Load up today' data

model_data = pd.get_dummies(model_data)
model_predict_data = pd.get_dummies(model_predict_data)

# Make sure all columns match

feature_list_model = set(list(model_data.columns))
feature_list_model_predict = set(list(model_predict_data.columns))

needed_columns = list(feature_list_model.difference(feature_list_model_predict))

for column in needed_columns:
    model_predict_data[column] = 0

feature_list_model = set(list(model_data.columns))
feature_list_model_predict = set(list(model_predict_data.columns))
huh_columns = list(feature_list_model_predict.difference(feature_list_model))

model_predict_data = model_predict_data.drop(huh_columns, axis=1)    

labels = np.array(model_data['total.runs'])
features = model_data.drop('total.runs', axis=1)
feature_list = list(features.columns)
features = np.array(features)

train_features, test_features, train_labels, test_labels = \
    train_test_split(features, labels, test_size=0.25, random_state=99)

forest = RandomForestRegressor(n_estimators=1000, random_state=99)

forest.fit(train_features, train_labels)

predictions = forest.predict(test_features)

errors = abs(predictions - test_labels)

print('Mean Absolute Error:', round(np.mean(errors), 2), 'runs.')

mape = 100 * (errors / test_labels)
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Mean Absolute Error: 3.76 runs.
Accuracy: 31.7 %.


In [5]:
predict_features = model_predict_data.drop('total.runs', axis=1)
predictions_today = forest.predict(predict_features)
predictions_today

array([11.063, 11.67 , 11.901, 11.285, 11.581, 12.663, 12.129, 11.74 ,
       11.653, 11.896])

In [6]:
train_labels

array([ 3,  1, 13, ...,  4, 12,  7])

In [10]:
np.mean(train_labels)

8.790876777251185

In [11]:
np.mean(predictions_today)

11.7581

In [12]:
data

,game.away.abbr,game.away.loss,game.away.probable_pitcher.era,game.away.probable_pitcher.id,game.away.probable_pitcher.loss,game.away.probable_pitcher.win,game.away.win,game.broadcast.network,game.day_night,game.home.abbr,game.home.loss,game.home.probable_pitcher.era,game.home.probable_pitcher.id,game.home.probable_pitcher.loss,game.home.probable_pitcher.win,game.home.win,game.venue.capacity,game.venue.field_orientation,game.venue.name,game.venue.stadium_type,game.venue.surface,game.weather.forecast.cloud_cover,game.weather.forecast.condition,game.weather.forecast.dew_point_f,game.weather.forecast.humidity,game.weather.forecast.temp_f,game.weather.forecast.wind.direction,game.weather.forecast.wind.speed_mph,home.win.pct,away.win.pct,total.runs
0,DET,6,2.500,8f131591-cc6e-43ad-a6e2-b15963fcb6ba,1.0,0.0,8,FS-N,D,MIN,4,2.177,56da7cb9-5975-46cd-8c63-96fce1245945,1.0,1.0,7,38649,E,Target Field,outdoor,grass,24.0,Cloudy,30.0,65.0,41.0,ENE,3.0,0.636364,0.571429,0
1,HOU,5,3.316,eebc991a-23ea-4f1c-ba3b-37ff21ee1603,2.0,0.0,10,SportsNet NW,D,SEA,4,3.156,4f914367-3a65-42d2-a6c4-28d9614e256f,0.0,4.0,13,47943,NE,T-Mobile Park,retractable,grass,87.0,Rain,39.0,66.0,50.0,SSW,6.0,0.764706,0.666667,0
2,COL,12,3.000,2764f896-603a-42bb-b031-f50893d42a63,1.0,1.0,3,NBCS-BA,D,SF,9,3.375,bbf735b4-af04-4ad8-b3e0-ea02dae770c4,1.0,1.0,7,41915,E,Oracle Park,outdoor,grass,11.0,Cloudy,50.0,68.0,61.0,W,11.0,0.437500,0.200000,0
3,MIL,5,4.235,0dbcd440-68fd-4e69-af97-8a7076f21eaf,1.0,2.0,10,SportsNet LA,D,LAD,8,3.780,b2fdb7e8-a700-408c-b238-a077165c4745,1.0,0.0,8,56000,NE,Dodger Stadium,outdoor,grass,70.0,Cloudy,37.0,29.0,72.0,SSW,6.0,0.500000,0.666667,0
8,PHI,5,3.000,4ec0aafd-a982-4521-b963-b26371dc8327,0.0,0.0,8,FS-FL,D,MIA,11,9.220,6fd307e1-8322-4f8c-9e3f-f71cd1ac9698,3.0,0.0,4,36742,SE,Marlins Park,retractable,grass,54.0,Cloudy,70.0,60.0,85.0,SSE,15.0,0.266667,0.615385,0
10,CWS,9,3.375,b5fe445d-6ac4-4542-a73c-69afced32a97,2.0,1.0,4,MLB Network,D,NYY,8,1.473,fdfda40f-e77b-4cc2-a72c-11951460beda,0.0,1.0,6,47309,E,Yankee Stadium,outdoor,grass,100.0,Cloudy,59.0,90.0,62.0,SSE,8.0,0.428571,0.307692,0
11,BAL,9,2.077,d6c41179-8aef-407c-843d-225a397935f0,1.0,1.0,6,NESN,D,BOS,10,6.000,875b903a-aabd-424e-b70e-1aa46a7334bc,1.0,0.0,5,37731,NE,Fenway Park,outdoor,grass,80.0,Rain,48.0,76.0,55.0,ESE,5.0,0.333333,0.400000,0
12,TB,4,2.250,ff772241-8fdd-488c-a81e-49b44ce600fc,0.0,2.0,11,SN1,D,TOR,10,2.411,cc86d4d3-1618-415e-b7fc-a303f3b8dd6f,2.0,0.0,5,53506,N,Rogers Centre,retractable,turf,100.0,Cloudy,35.0,85.0,40.0,ENE,19.0,0.333333,0.733333,0
13,OAK,8,2.500,79551d40-c562-4e2b-8692-9fa7730572b7,0.0,3.0,10,FS-SW,D,TEX,7,1.862,84fa497c-4947-48d9-85ed-352c1e5ab113,1.0,0.0,6,49115,SE,Globe Life Park in Arlington,outdoor,grass,0.0,Sunny,34.0,30.0,66.0,NW,9.0,0.461538,0.555556,0
14,PIT,6,3.000,0b4c4345-7609-4ab1-9aaa-41d56d7f1fda,2.0,0.0,7,MASN2,D,WSH,6,3.316,2527770b-cf48-42b9-81fa-9323756fb311,2.0,1.0,7,41313,N,Nationals Park,outdoor,grass,100.0,Storm,64.0,77.0,72.0,SSW,12.0,0.538462,0.538462,0
